In [2]:
import tensorflow as tf
import numpy as np

In [24]:
n=5
m=7
x = tf.random.uniform(shape=[m,1])
W = tf.random.uniform(shape=[n,m])
y = tf.random.uniform(shape=[n])
diag_y = tf.linalg.diag(y)
print(y,diag_y)
u = tf.random.uniform(shape=[1,n])
f= u@diag_y@W@x
print(f)

tf.Tensor([0.03272021 0.28152406 0.91766584 0.22948515 0.45178938], shape=(5,), dtype=float32) tf.Tensor(
[[0.03272021 0.         0.         0.         0.        ]
 [0.         0.28152406 0.         0.         0.        ]
 [0.         0.         0.91766584 0.         0.        ]
 [0.         0.         0.         0.22948515 0.        ]
 [0.         0.         0.         0.         0.45178938]], shape=(5, 5), dtype=float32)
tf.Tensor([[1.5011142]], shape=(1, 1), dtype=float32)
